# ✅ JSON Instagram Template Editor (v3) — Pillow
Works with templates using **rgba() colors**, **image elements**, **center (x,y)** coordinates, **objectFit**, and **fontFamily**. Also supports simple **placeholders** like `{{title}}`, `{{description}}`, `{{imageUrl}}`.

**Steps**
1) Install & set up
2) Paste JSON + placeholders → Parse
3) Render preview
4) Export updated JSON

Tip: Upload custom fonts to a `fonts/` folder (e.g., `LeagueGothic-Regular.ttf`).

In [1]:
#@title 1) Install
!pip -q install pillow requests ipywidgets rich
from google.colab import output  # type: ignore
output.enable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00


In [2]:
#@title 2) Imports & helpers
import json, os, io, re, requests
from typing import Any, Dict, List
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont, ImageOps, ImageColor

FONTS_DIR = 'fonts'
DEFAULT_TTF = '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'

def get_font(font_family: str|None, font_size: int):
    candidates = []
    if font_family: candidates.append(os.path.join(FONTS_DIR, font_family))
    candidates.append(DEFAULT_TTF)
    for p in candidates:
        try:
            if os.path.exists(p):
                return ImageFont.truetype(p, font_size)
        except Exception:
            pass
    return ImageFont.load_default()

def parse_color(value: Any, default=(0,0,0,255)):
    # Accept '#RRGGBB', '#RGB', 'rgba(r,g,b,a)', 'rgb(r,g,b)', tuples/lists, or named colors.
    if value is None: return default
    if isinstance(value, (list, tuple)):
        if len(value)==3: return (int(value[0]), int(value[1]), int(value[2]), 255)
        if len(value)==4: return (int(value[0]), int(value[1]), int(value[2]), int(value[3]))
    if isinstance(value, str):
        v = value.strip()
        if v.startswith('#'):
            v = v.lstrip('#')
            if len(v)==3:
                r,g,b = [int(c*2,16) for c in v]
                return (r,g,b,255)
            if len(v)==6:
                r=int(v[0:2],16); g=int(v[2:4],16); b=int(v[4:6],16)
                return (r,g,b,255)
        import re
        m = re.match(r'rgba?\(([^)]+)\)', v, re.I)
        if m:
            parts = [p.strip() for p in m.group(1).split(',')]
            r = int(float(parts[0])); g = int(float(parts[1])); b = int(float(parts[2]))
            a = 255
            if len(parts)==4:
                a_val = float(parts[3])
                a = int(a_val*255) if 0 <= a_val <= 1 else int(a_val)
            return (r,g,b,a)
        try:
            rgb = ImageColor.getrgb(v)
            return (rgb[0], rgb[1], rgb[2], 255)
        except Exception:
            return default
    return default

def load_image(src: str, size=None, fit='cover'):
    if isinstance(src, str) and (src.startswith('http://') or src.startswith('https://')):
        import requests, io
        r = requests.get(src, timeout=20)
        r.raise_for_status()
        im = Image.open(io.BytesIO(r.content)).convert('RGBA')
    else:
        im = Image.open(src).convert('RGBA')
    if size is None: return im
    if fit.lower() == 'cover':
        return ImageOps.fit(im, size, Image.LANCZOS)
    bg = Image.new('RGBA', size, (0,0,0,0))
    im = im.copy(); im.thumbnail(size, Image.LANCZOS)
    x = (size[0]-im.width)//2; y = (size[1]-im.height)//2
    bg.paste(im, (x,y), im)
    return bg

def interpolate_placeholders(obj, values: Dict[str,str]):
    if isinstance(obj, dict):
        return {k: interpolate_placeholders(v, values) for k,v in obj.items()}
    if isinstance(obj, list):
        return [interpolate_placeholders(v, values) for v in obj]
    if isinstance(obj, str):
        out = obj
        for key, val in values.items():
            out = out.replace('{{'+key+'}}', val)
        return out
    return obj


In [3]:
#@title 3) Renderer (text, image, shape-rectangle) with center coords
def render(template: Dict[str,Any]):
    width  = int(template.get('width', 1080))
    height = int(template.get('height', 1350))
    bgc = parse_color(template.get('backgroundColor') or template.get('background_color') or '#FFFFFF')
    canvas = Image.new('RGBA', (width, height), bgc)

    layers = []
    for el in template.get('elements', []):
        z = int(el.get('zIndex') or el.get('z_index') or 0)
        layers.append((z, el))
    layers.sort(key=lambda t: t[0])

    draw = ImageDraw.Draw(canvas)
    for _, el in layers:
        etype = (el.get('type') or '').lower()

        if etype == 'shape' and (el.get('shapeType') or '').lower()=='rectangle':
            x = float(el.get('x', 0)); y = float(el.get('y', 0))
            w = int(el.get('width', 0)); h = int(el.get('height', 0))
            color = parse_color(el.get('color'), (0,0,0,255))
            rect = Image.new('RGBA', (w, h), color)
            canvas.alpha_composite(rect, dest=(int(x - w/2), int(y - h/2)))

        elif etype == 'image':
            src = el.get('content') or el.get('src')
            if not src: continue
            x = float(el.get('x', 0)); y = float(el.get('y', 0))
            w = int(el.get('width', 0)); h = int(el.get('height', 0))
            fit = (el.get('objectFit') or el.get('fit') or 'cover').lower()
            opacity = el.get('opacity')
            if opacity is None: alpha = 255
            else:
                try:
                    f = float(opacity)
                    alpha = int(f*255) if 0 <= f <= 1 else int(f)
                except:
                    alpha = 255
            try:
                im = load_image(src, (w,h), fit)
                if alpha < 255:
                    im = im.copy(); im.putalpha(alpha)
                canvas.alpha_composite(im, dest=(int(x - w/2), int(y - h/2)))
            except Exception as e:
                print('[image] error:', e)

        elif etype == 'text':
            text = str(el.get('content') or el.get('text') or '')
            x = float(el.get('x', 0)); y = float(el.get('y', 0))
            w = int(el.get('width', 0)); h = int(el.get('height', 0))
            color = parse_color(el.get('color', '#000'))
            fs = int(el.get('fontSize', 40)); ff = el.get('fontFamily') or el.get('font_family')
            align = (el.get('textAlign') or 'left').lower()
            valign = (el.get('verticalAlign') or 'top').lower()
            font = get_font(ff, fs)

            def wrap(draw, text, font, max_w):
                if not text: return ''
                words = text.split()
                lines, cur = [], ''
                for w_ in words:
                    test = w_ if not cur else f"{cur} {w_}"
                    bbox = draw.textbbox((0,0), test, font=font)
                    if bbox[2] <= max_w:
                        cur = test
                    else:
                        if cur: lines.append(cur)
                        cur = w_
                if cur: lines.append(cur)
                return '\n'.join(lines)

            wrapped = wrap(draw, text, font, w)
            lines = wrapped.split('\n') if wrapped else ['']
            line_heights = [(draw.textbbox((0,0), ln, font=font)[3]) for ln in lines]
            total_h = sum(line_heights) + 4*(len(lines)-1)

            box_left = int(x - w/2); box_top = int(y - h/2)
            if valign == 'middle':
                yy = box_top + (h - total_h)//2
            elif valign == 'bottom':
                yy = box_top + (h - total_h)
            else:
                yy = box_top

            for ln in lines:
                bbox = draw.textbbox((0,0), ln, font=font)
                lw = bbox[2]-bbox[0]
                if align == 'center':
                    xx = box_left + (w - lw)//2
                elif align == 'right':
                    xx = box_left + (w - lw)
                else:
                    xx = box_left
                draw.text((xx, yy), ln, font=font, fill=color)
                yy += (bbox[3]-bbox[1]) + 4
    return canvas.convert('RGB')


In [4]:
#@title 4) UI — Paste JSON, placeholders, render, export
import ipywidgets as W

json_in = W.Textarea(value='', placeholder='Paste your template JSON here…', layout=W.Layout(width='100%', height='260px'))
title_in = W.Text(value='Your Big Title', description='title:', layout=W.Layout(width='100%'))
desc_in  = W.Text(value='A short description goes here.', description='description:', layout=W.Layout(width='100%'))
img_in   = W.Text(value='https://images.unsplash.com/photo-1475724017904-b712052c192a?q=80&w=1200&auto=format&fit=crop', description='imageUrl:', layout=W.Layout(width='100%'))

parse_btn = W.Button(description='Parse & Render', button_style='primary')
img_out = W.Output(); status_out = W.Output()
export_btn = W.Button(description='Export JSON')
export_out = W.Output()

state = {'template': None, 'rendered': None}

def do_parse(_=None):
    status_out.clear_output(); img_out.clear_output()
    try:
        tpl = json.loads(json_in.value)
        values = {'title': title_in.value, 'description': desc_in.value, 'imageUrl': img_in.value}
        tpl2 = interpolate_placeholders(tpl, values)
        state['template'] = tpl2
        im = render(tpl2); state['rendered'] = im
        with img_out: display(im)
        with status_out: print('✅ Parsed and rendered.')
    except Exception as e:
        with status_out: print('❌ Error:', e)

def do_export(_=None):
    export_out.clear_output()
    if not state['template']:
        with export_out: print('No template loaded.')
        return
    txt = json.dumps(state['template'], indent=2)
    with open('updated_template.json','w') as f: f.write(txt)
    with export_out:
        print('Updated JSON:\n')
        print(txt)
        print('\nSaved as updated_template.json (download from Files sidebar).')

parse_btn.on_click(do_parse)
export_btn.on_click(do_export)

ui = W.VBox([
    W.HTML('<h3>Paste Template JSON</h3>'), json_in,
    W.HTML('<h3>Placeholders</h3>'), title_in, desc_in, img_in,
    parse_btn, status_out,
    W.HTML('<h3>Preview</h3>'), img_out,
    W.HTML('<h3>Export</h3>'), export_btn, export_out
])
ui
